In [25]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Process the data and convert it into a pandas DataFrame
data = []
with open("processed_parallel_tracking.txt", "r") as file:
    for line in file:
        values = line.strip().split()
        frame = float(values[0])
        pedestrian_id = float(values[1])
        x = float(values[-4])
        y = float(values[-2])
        data.append([frame, pedestrian_id, x, y])

df = pd.DataFrame(data, columns=["frame", "pedestrian_id", "x", "y"])


In [19]:
# count amount of each value in the pedestrian_id column


pedestrian_id
1.0    54
Name: count, dtype: int64

In [26]:

import numpy as np

def polynomial_func(x, a, b):
    return a * x + b

def predict_trajectory(past_positions, future_steps=12):
    # Number of past timesteps
    past_steps = len(past_positions)
    
    # Prepare the input features (time steps)
    t = np.array(range(past_steps))
    
    # Fit the polynomial curve
    coeffs, _ = curve_fit(polynomial_func, t, past_positions)
    
    # Predict future positions
    future_time_steps = np.array(range(past_steps, past_steps + future_steps))
    future_positions = polynomial_func(future_time_steps, *coeffs)
    
    return future_positions

# compute ade and fde of predicted trajectory x and y compared to gt_future_x and gt_future_y
def compute_ade_fde(predict_future_x, pred_future_y, gt_future_x, gt_future_y):
    ade = np.mean(np.sqrt((predict_future_x - gt_future_x)**2 + (pred_future_y - gt_future_y)**2))
    fde = np.sqrt((predict_future_x[-1] - gt_future_x[-1])**2 + (pred_future_y[-1] - gt_future_y[-1])**2)
    return ade, fde

past_steps = 8
future_steps = 12
# create past_x data and past_y data and gt_future_x and gt_future_y for all frames which have 8 past and 12 future steps
max_frame = int(df["frame"].max())
total_ade = 0
total_fde = 0
for i in range(past_steps, max_frame - future_steps):
    past_x = df[df["pedestrian_id"] == 1]["x"].values[i-past_steps:i]
    past_y = df[df["pedestrian_id"] == 1]["y"].values[i-past_steps:i]
    gt_future_x = df[df["pedestrian_id"] == 1]["x"].values[i:i+future_steps]
    gt_future_y = df[df["pedestrian_id"] == 1]["y"].values[i:i+future_steps]

    predicted_trajectory_x = predict_trajectory(past_x)
    predicted_trajectory_y = predict_trajectory(past_y)

    ade, fde = compute_ade_fde(predicted_trajectory_x, predicted_trajectory_y, gt_future_x, gt_future_y)
    total_ade += ade
    total_fde += fde
# divide by the number of frames to get the average
average_ade =  total_ade / (max_frame - 20)
average_fde = total_fde / (max_frame - 20)
print(f"ade is {average_ade} while fde is {average_fde}")





    

ade is 0.919453016208007 while fde is 1.6762692876586454


In [13]:

import numpy as np

def polynomial_func(x, a, b):
    return a * x + b

def predict_trajectory(past_x, past_y, future_steps=12):
    # Number of past timesteps
    past_steps = len(past_x)
    
    # Prepare the input features (time steps)
    t = np.array(range(past_steps))
    
    # Fit the polynomial curve
    t_to_x_coeffs, _ = curve_fit(polynomial_func, t, past_x)
    
    # Predict future positions
    future_time_steps = np.array(range(past_steps, past_steps + future_steps))
    future_x_positions = polynomial_func(future_time_steps, *t_to_x_coeffs)
    # use future x positions to predict future y positions
    x_to_y_coeffs, _ = curve_fit(polynomial_func, past_x, past_y)
    future_y_positions = polynomial_func(future_x_positions, *x_to_y_coeffs)
    
    return future_x_positions, future_y_positions





# create past_x data and past_y data and gt_future_x and gt_future_y for all frames which have 8 past and 12 future steps
max_frame = int(df["frame"].max())
total_ade = 0
total_fde = 0
for i in range(7, max_frame - 13):
    past_x = df[df["pedestrian_id"] == 1]["x"].values[i-7:i+1]
    past_y = df[df["pedestrian_id"] == 1]["y"].values[i-7:i+1]
    gt_future_x = df[df["pedestrian_id"] == 1]["x"].values[i+1:i+13]
    gt_future_y = df[df["pedestrian_id"] == 1]["y"].values[i+1:i+13]

    predicted_trajectory_x, predicted_trajectory_y = predict_trajectory(past_x, past_y)

    ade, fde = compute_ade_fde(predicted_trajectory_x, predicted_trajectory_y, gt_future_x, gt_future_y)
    total_ade += ade
    total_fde += fde
# divide by the number of frames to get the average
average_ade =  total_ade / (max_frame - 20)
average_fde = total_fde / (max_frame - 20)
print(f"ade is {average_ade} while fde is {average_fde}")

ade is 0.9525972394048189 while fde is 1.5149965574095012


In [14]:
import numpy as np

def polynomial_func(x, a, b):
    return a * x + b

def predict_trajectory(past_x, past_y, future_steps=12):
    # Number of past timesteps
    past_steps = len(past_x)
    
    # Prepare the input features (time steps)
    t = np.array(range(past_steps))
    
    # Fit the polynomial curve
    t_to_y_coeffs, _ = curve_fit(polynomial_func, t, past_y)
    
    # Predict future positions
    future_time_steps = np.array(range(past_steps, past_steps + future_steps))
    future_y_positions = polynomial_func(future_time_steps, *t_to_y_coeffs)
    # use future x positions to predict future y positions
    y_to_x_coeffs, _ = curve_fit(polynomial_func, past_y, past_x)
    future_x_positions = polynomial_func(future_y_positions, *y_to_x_coeffs)
    
    return future_x_positions, future_y_positions

# create past_x data and past_y data and gt_future_x and gt_future_y for all frames which have 8 past and 12 future steps
max_frame = int(df["frame"].max())
total_ade = 0
total_fde = 0
for i in range(7, max_frame - 13):
    past_x = df[df["pedestrian_id"] == 1]["x"].values[i-7:i+1]
    past_y = df[df["pedestrian_id"] == 1]["y"].values[i-7:i+1]
    gt_future_x = df[df["pedestrian_id"] == 1]["x"].values[i+1:i+13]
    gt_future_y = df[df["pedestrian_id"] == 1]["y"].values[i+1:i+13]

    predicted_trajectory_x, predicted_trajectory_y = predict_trajectory(past_x, past_y)

    ade, fde = compute_ade_fde(predicted_trajectory_x, predicted_trajectory_y, gt_future_x, gt_future_y)
    total_ade += ade
    total_fde += fde
# divide by the number of frames to get the average
average_ade =  total_ade / (max_frame - 20)
average_fde = total_fde / (max_frame - 20)
print(f"ade is {average_ade} while fde is {average_fde}")

ade is 0.887081115053536 while fde is 1.6269195383430344


In [5]:
import numpy as np
from scipy.interpolate import CubicSpline

# Create sample pedestrian trajectory data
trajectory_data = np.array([[0.0, 0.0], [1.0, 2.0], [2.0, 3.0], [3.0, 1.0], [4.0, 2.0]])

# Extract time steps and positions from the trajectory data
time_steps = trajectory_data[:, 0]
x_positions = trajectory_data[:, 1]

# Create a cubic spline interpolation function
spline = CubicSpline(time_steps, x_positions)

# Generate future time steps for prediction
num_predictions = 5
future_time_steps = np.arange(time_steps[-1] + 1, time_steps[-1] + num_predictions + 1)

# Predict future positions using the spline interpolation
predicted_positions = spline(future_time_steps)

# Print the predicted trajectory
print("Predicted trajectory:")
print(np.column_stack((future_time_steps, predicted_positions)))

Predicted trajectory:
[[  5.  14.]
 [  6.  45.]
 [  7. 103.]
 [  8. 196.]
 [  9. 332.]]
